## 사진을 보고 그림 으로 바꾸어주는 모델! 
### 필요한 모듈들 먼저 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/bootcamp/cp01/project/

/content/drive/MyDrive/bootcamp/cp01/project


In [3]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-_0y3oxsj
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-_0y3oxsj
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=0e4465b487d799e221446bac79b8ae09134fcd6120ebc964154ab271e84b26ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-k8wz6klk/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [4]:
!pip install imageio
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2 MB 1.2 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc3 3.11.4 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
plotnine 0.6.0 requires scipy>=1.2.0, but you have scipy 1.1.0 which is incompatible.
jax 0.2.25 requires scipy>=1.2.1, but you have scipy 1.1.0 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [5]:
import os
import matplotlib.pyplot as plt


from model.cycle_gan import CycleGAN
from utils.loaders import DataLoader
# from utils.file_split import data_split


## Data split & Folder setting

In [ ]:
import random
import os
import shutil

A_path= '/content/drive/MyDrive/bootcamp/cp01/project/data'
B_path = '/content/drive/MyDrive/bootcamp/cp01/project/data/thumbnails128x128'

temp_file_path =[]
def search(dirname):
    try:
        filenames = os.listdir(dirname)
        for filename in filenames:
            full_filename = os.path.join(dirname, filename)
            if os.path.isdir(full_filename):
                search(full_filename)
            else:
                ext = os.path.splitext(full_filename)[-1]
                if ext == '.png': 
                    temp_file_path.append(full_filename)
    except PermissionError:
        pass
    return temp_file_path


def data_split(temp1,temp2,test_len):
    dataA_dir = temp1
    dataB_dir = temp2

    len_dirA = int(len(dataA_dir))
    len_dirB = int(len(dataB_dir))

    train_len = min(len_dirA,len_dirB)
    
    test_A_dir = random.sample(dataA_dir,test_len)
    test_B_dir = random.sample(dataB_dir,test_len)


    for del_value in test_A_dir:
        dataA_dir.remove(del_value) 
    train_A_dir = random.sample(dataA_dir,train_len-test_len)
    
    for del_value in test_B_dir:
        dataB_dir.remove(del_value)
    train_B_dir = random.sample(dataB_dir,train_len-test_len)



    # 디렉토리 로 파일 옮겨주면서 분류
    # A는 fullpath
    base_path = './data/'
    trainA_path = base_path+"trainA/"
    for i in train_A_dir:
        temp = i.split('/')
        try:
            shutil.copyfile(i,trainA_path+temp[-1])
        except IsADirectoryError:
            pass
    # B는 /data 안에 담겨있다.
    trainB_path = base_path+"trainB/"
    for i in train_B_dir:
        try:
            shutil.copyfile("./data/"+i,trainB_path+i)
        except IsADirectoryError:
            pass

    testA_path = base_path+"testA/"
    for i in test_A_dir:
        temp.clear()
        temp = i.split('/')
        try:
            shutil.copyfile(i,testA_path+temp[-1])
        except IsADirectoryError:
            pass            

    testB_path = base_path+"testB/"
    for i in test_B_dir:
        try:
            shutil.copyfile(str("./data/"+i),testB_path+i) 
        except IsADirectoryError:
            pass



temp1 = os.listdir(A_path)
temp2 = search(B_path)
data_split(temp2,temp1,100)

In [6]:
# run params
DATA_NAME = 'drawing'
DATA_NAME2 = 'face_drawing'
RUN_FOLDER = './run/{}/'.format(DATA_NAME)
RUN_FOLDER += '_'.join([DATA_NAME2])
print(RUN_FOLDER)
if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' # 'build' # 

./run/drawing/face_drawing


## Define Model

In [7]:
IMAGE_SIZE=256
data_loader=DataLoader(img_res=(IMAGE_SIZE,IMAGE_SIZE))

In [8]:
gan_r = CycleGAN(
    input_dim=(IMAGE_SIZE,IMAGE_SIZE,3)
    ,learning_rate=0.0002
    ,lambda_validation=1
    ,lambda_reconstr = 10
    ,lambda_id=5
    ,generator_type='r'
    ,gen_n_filters=32
    ,disc_n_filters=64
    )
if mode =="build":
    gan_r.save_model(RUN_FOLDER)
else:
    # 이미 학습된게 있으면 불러오기
    gan_r.load_weights(os.path.join(RUN_FOLDER,'weights/weights.h5'))


INFO:tensorflow:Assets written to: ram://9bb52d37-adf5-4a55-ae87-0c7368d347e4/assets
INFO:tensorflow:Assets written to: ram://7218c8ae-a591-40b4-9cd3-5d8348331c59/assets
INFO:tensorflow:Assets written to: ram://54e3f6bd-8415-4d4d-90ed-04210ad190be/assets
INFO:tensorflow:Assets written to: ram://099e7b6f-0515-4f0c-8ccc-928e0ba2f042/assets
INFO:tensorflow:Assets written to: ram://9353cae6-4044-4145-a6df-26b2e2ceae72/assets


In [9]:
gan_r.g_BA.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 reflection_padding2d_20 (Refle  (None, 262, 262, 3)  0          ['input_4[0][0]']                
 ctionPadding2D)                                                                                  
                                                                                                  
 conv2d_32 (Conv2D)             (None, 256, 256, 32  4736        ['reflection_padding2d_20[0][0]']
                                )                                                           

In [10]:
gan_r.g_AB.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 reflection_padding2d (Reflecti  (None, 262, 262, 3)  0          ['input_3[0][0]']                
 onPadding2D)                                                                                     
                                                                                                  
 conv2d_10 (Conv2D)             (None, 256, 256, 32  4736        ['reflection_padding2d[0][0]']   
                                )                                                           

In [11]:
gan_r.d_A.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      3136      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       131200    
                                                                 
 instance_normalization (Ins  (None, 64, 64, 128)      0         
 tanceNormalization)                                             
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                             

In [12]:
gan_r.d_B.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 128, 128, 64)      3136      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 128, 128, 64)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 64, 64, 128)       131200    
                                                                 
 instance_normalization_3 (I  (None, 64, 64, 128)      0         
 nstanceNormalization)                                           
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                           

In [15]:
import random
import os 
BATCH_SIZE=1
EPOCHS=3
PRINT_EVERY_N_BATCHE=10

TEST_A_FILE = random.choice(os.listdir("./data/testA"))
TEST_B_FILE = random.choice(os.listdir("./data/testB"))

In [ ]:
gan_r.train(data_loader
            ,run_folder=RUN_FOLDER
            ,epochs=EPOCHS
            ,test_A_file=TEST_A_FILE
            ,test_B_file=TEST_B_FILE
            ,batch_size=BATCH_SIZE
            ,sample_interval=PRINT_EVERY_N_BATCHE)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
INFO:tensorflow:Assets written to: ram://851c5292-c873-4500-ab6e-70367b9e9ac4/assets
INFO:tensorflow:Assets written to: ram://c5a3f570-3fcf-4f0a-8ddd-d0373767f785/assets
INFO:tensorflow:Assets written to: ram://d70fd305-e90e-44cd-92b7-1be581c925d3/assets
INFO:tensorflow:Assets written to: ram://4f162156-620f-4d64-8e7b-491d96cf7140/assets
(256, 256, 3) (256, 256, 3)
(256, 256, 3) (256, 256, 3)
(256, 256, 3) (256, 256, 3)
(256, 256, 3) (256, 256, 3)
(256, 256, 3) (256, 256, 3)
(256, 256, 3) (256, 256, 3)
INFO:tensorflow:Assets written to: ram://187a513d-9a4b-4cef-91b9-fa4c34742581/assets
INFO:tensorflow:Assets written to: ram://f9a17fab-76ac-416b-8986-5bae84cde5a3/assets
INFO:tensorflow:Assets written to: ram://39a48852-dee5-4381-84d6-d95d444b571a/assets
INFO:tensorflow:Assets written to: ram://b245bb20-7095-4054-b8f7-b749496b493f/assets
INFO:tensorflow:Assets written to: ram://59f42e3b-364e-4f63-83ac-9f1f5249d2e3/assets
(256, 256, 3) (256, 256, 3)
(25